In [178]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from glob import glob
from PIL import Image
import os
import random
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [179]:


#Model
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPooling2D,MaxPool2D,AveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
from tensorflow.keras.metrics import PrecisionAtRecall,Recall



In [180]:
tf.config.list_physical_devices("GPU")

In [181]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [182]:
from numpy.random import seed
seed(0)

In [183]:
tf.random.set_seed(0)

In [184]:
path = '../input/chest-xray-covid19-pneumonia/Data/train'

diag_code_dict = {
    'COVID19': 0,
    'NORMAL': 1,
    'PNEUMONIA': 2}

diag_title_dict = {
    'COVID19': 'Covid-19',
    'NORMAL': 'Healthy',
    'PNEUMONIA': 'Pneumonia'}

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(path, '*','*.png'))}

imageid_path_dict

In [185]:


test_path = '../input/chest-xray-covid19-pneumonia/Data/test/'
train_path = '../input/chest-xray-covid19-pneumonia/Data/train/'

classes = ["COVID19", "NORMAL", "PNEUMONIA"]
num_classes = len(classes)
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True, validation_split=0.001)

test_datagen = ImageDataGenerator(rescale=1./255,
                                 validation_split=0.999)

train_gen = train_datagen.flow_from_directory(directory=train_path,
                                             target_size=(299, 299),
                                             class_mode='categorical',
                                             subset='training',
                                             shuffle=True, classes=classes,
                                             batch_size=batch_size,
                                             )

test_gen = test_datagen.flow_from_directory(directory=test_path,
                                             target_size=(299, 299),
                                             class_mode='categorical',
                                             subset='validation',
                                             shuffle=False, classes=classes,
                                             batch_size=batch_size,
                                             )

In [186]:
# model = Sequential()
# model.add(Conv2D(32, (3,3), activation='relu', padding='Same', input_shape=(299, 299, 1)))
# model.add(BatchNormalization())

# ################

# model.add(Conv2D(64, (3,3), activation='relu', padding='Same'))
# model.add(BatchNormalization())
# model.add(AveragePooling2D(pool_size = (2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3,3), activation='relu', padding='Same'))
# model.add(BatchNormalization())

# model.add(Conv2D(64, (3,3), activation='relu', padding='Same'))
# model.add(BatchNormalization())
# model.add(AveragePooling2D(pool_size = (2,2)))
# model.add(Dropout(0.25))

# #################

# model.add(Conv2D(64, (3,3), activation='relu', padding='Same'))
# model.add(BatchNormalization())
# model.add(AveragePooling2D(pool_size = (2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3,3), activation='relu', padding='Same'))
# model.add(BatchNormalization())

# model.add(Conv2D(64, (3,3), activation='relu', padding='Same'))
# model.add(BatchNormalization())
# model.add(AveragePooling2D(pool_size = (2,2)))
# model.add(Dropout(0.25))

# #################

# model.add(Flatten())

# model.add(BatchNormalization())
# model.add(Dense(128, activation='relu'))
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

# #Output 
# model.add(BatchNormalization())
# model.add(Dense(num_classes, activation='softmax'))

In [187]:
transfer_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False)

In [188]:
model = Sequential([
    transfer_model, 
    Conv2D(64, (3,3), activation='relu', padding='Same'),
    BatchNormalization(),
    AveragePooling2D(pool_size = (2,2)),
    Dropout(0.25),
    Flatten(), 
    BatchNormalization(),
    Dense(128, activation='relu'), 
    Dropout(0.25),
    BatchNormalization(),
    Dense(num_classes, activation='softmax')
])

In [189]:
model.summary()

In [190]:
opt = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

In [191]:
from tensorflow import keras

keras.utils.plot_model(model)

In [192]:
epochs=30

#callback

history = model.fit(train_gen,
                    steps_per_epoch=len(train_gen) // batch_size,
                    validation_steps=len(test_gen) // batch_size,
                    validation_data=test_gen, epochs=epochs)

In [193]:
y_pred = model.predict(test_gen)

In [194]:
history.history.keys()

In [195]:
fig, axarr = plt.subplots(1,2, figsize=(15,5), sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x = historyDF.index, y = history.history['acc'], ax=axarr[0], label="Training");
ax = sns.lineplot(x = historyDF.index, y = history.history['val_acc'], ax=axarr[0], label="Validation");
ax.set_ylabel('Accuracy')

ax = sns.lineplot(x = historyDF.index, y = history.history['loss'], ax=axarr[1], label="Training");
ax = sns.lineplot(x = historyDF.index, y = history.history['val_loss'], ax=axarr[1], label="Validation");
ax.set_ylabel('Loss')


axarr[0].set_title('Training and Validation metric recall')
axarr[1].set_title('Training and Validation loss')

for ax in axarr:
    ax.set_xlabel('Epochs')

plt.suptitle('Training and performance plots', fontsize=16, weight='bold');
fig.tight_layout(pad=3.0)
plt.show()

In [203]:
model.save('xception(30last).h5')

In [201]:
predictions = np.array(list(map(lambda x: np.argmax(x), y_pred)))

y_true=test_gen.classes

CMatrix = pd.DataFrame(confusion_matrix(y_true, predictions), columns=classes, index =classes)

plt.figure(figsize=(12, 6))
ax = sns.heatmap(CMatrix, annot = True, fmt = 'g' ,vmin = 0, vmax = 250,cmap = 'Blues')
ax.set_xlabel('Predicted',fontsize = 14,weight = 'bold')
ax.set_xticklabels(ax.get_xticklabels(),rotation =0);

ax.set_ylabel('Actual',fontsize = 14,weight = 'bold') 
ax.set_yticklabels(ax.get_yticklabels(),rotation =0);
ax.set_title('Confusion Matrix - Test Set',fontsize = 16,weight = 'bold',pad=20);

In [198]:
print(classification_report(y_true, predictions))

In [199]:
acc = accuracy_score(y_true, predictions)

results_all = precision_recall_fscore_support(y_true, predictions, average='macro', zero_division=1)
results_class = precision_recall_fscore_support(y_true, predictions, average=None, zero_division=1)

metric_columns = ['Precision','Recall', 'F1-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['COVID','Normal', 'Viral Pneumonia','Total']

def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');

metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F1-Score')#Results by Class
